In [38]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Notebook conversion

In [9]:
import nbformat as nb
import nbformat.v4.nbbase as nb4
from nbconvert import RSTExporter

def convert_notebook(nbfile, rstfile):
    (rst, resources) = rstexporter.from_filename(nbfile)
    with open(rstfile, 'w') as fout:
        fout.write(rst)
        
rstexporter = RSTExporter()
rstexporter.template_file = './templates/rst.tpl'

convert_notebook('ulab-manual.ipynb', './source/ulab-manual.rst')

In [10]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [11]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../../micropython/ports/unix/micropython", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        else:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

In [ ]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [ ]:
pyb.exit_raw_repl()
pyb.close()

In [ ]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return timeit

# Introduction

In https://micropython-usermod.readthedocs.io/en/latest/usermods_14.html, I mentioned that I have another story in store, for another day. The day has come, so here is my story.

## ulab

`ulab` is a numpy-like module for micropython, meant to simplify and speed up common mathematical operations on. The primary goal was to implement a small subset of numpy that might be useful in the context of a microcontroller. This means low-level data processing of linear (array) and two-dimensional (matrix) data.

## Purpose

Of course, the first question that one has to answer is, why on Earth one would need a fast math library on a microcontroller. After all, it is not expected that heavy number crunching is going to take place on bare metal. It is not meant to. On a PC, the main reason for writing fast code is the sheer amount of data that one wants to process. On a microcontroller, the data volume is probably small, but it might lead to catastrophic system failure, if these data are not processed in time, because the microcontroller is supposed to interact with the outside world in a timely fashion. In fact, this latter objective was the initiator of this project: I needed the Fourier transform of the ADC signal, and all the available options were simply too slow. 

In addition to speed, another issue that one has to keep in mind when working with embedded systems is the amount of available RAM: I believe, everything here could be implemented in pure python with relatively little effort, but the price we would have to pay for that is not only speed, but RAM, too. python code, if is not frozen, and compiled into the firmware, has to be compiled at runtime, which is not exactly a cheap process. On top of that, if numbers are stored in a list or tuple, which would be the high-level container, then they occupy 8 bytes, no matter, whether they are all smaller than 100, or larger than one hundred million. This is obviously a waste of resources in an environment, where resources are scarce. 

Finally, there is a reason for using micropython in the first place. Namely, that a microcontroller can be programmed in a very elegant, and *pythonic* way. But if it is so, why should we not extend this idea to other tasks and concepts that might come up in this context? If there was no other reason than this *elegance*, I would find that convincing enough.

Based on the above-mentioned considerations, all functions are implemented in a way that 

1. conforms to `numpy` as much as possible
2. is so frugal with RAM as possible,
3. and yet, fast. Much faster than pure python.

The main points of `ulab` are 

- compact, iterable and slicable container of numerical data in 1, and 2 dimensions (arrays and matrices). These containers support all the relevant unary and binary operators (e.g., `len`, ==, +, *, etc.)
- vectorised computations on micropython iterables and numerical arrays/matrices (in numpy-speak, universal functions)
- basic linear algebra routines (matrix inversion, multiplication, reshaping, and transposition)
- polynomial fits to numerical data
- fast Fourier transforms

At the time of writing this manual (for version 0.17), the library adds approximately 23 kB of extra compiled code to the firmware. 

## Resources and legal matters

The source code can be found under https://github.com/v923z/micropython-ulab/code/. The source of this user manual is under https://github.com/v923z/micropython-ulab/docs/manual/, while the technical details of the implementation are discussed at great length in https://github.com/v923z/micropython-ulab/docs/ulab.ipynb. If you want an even thorougher explanation on why the various constructs of the implementation work, and work in that particular way, you can read more on the subject under https://micropython-usermod.readthedocs.io/en/latest/, where I demonstrate, what you have to do, if you want to make a C object behave in a *pythonic* way. 

The MIT licence applies to all material. 

## Friendly request

If you use `ulab`, and bump into a bug, or think that a particular function is missing, or its behaviour does not conform to `numpy`, please, raise an issue on github, so that the community can profit from your experiences. 

Even better, if you find the project useful, and think that it could be made better, faster, smaller, and shinier, please, consider contributing, and issue a pull request with the implementation of your improvements and new features. `ulab` can only become successful, if it offers what the community needs.

These last comments apply to the documentation, too. If, in your opinion, the documentation is obscure, misleading, or not detailed enough, please, let me know, so that *we* can fix it.

# ndarray, the basic container

The `ndarray` is the underlying container of numerical data. It is derived from micropython's own `array` object, but has a great number of extra features starting with how it can be initialised, how operations can be done on it, and which functions can accept it as an argument.

Since `ndarray`s are a binary container, they are also compact, meaning that they take only a couple of bytes of extra RAM in addition to what is required for storing the numbers themselves. `ndarray`s are also type-aware, i.e., one can save RAM by specifying a data type, and using the smallest possible type. Five such types are defined, namely `uint8`, `int8`, which occupy a single byte of memory per datum, `uint16`, and `int16`, which occupy two bytes per datum, and `float`, which occupies four bytes per datum. 

On the following pages, we will see how one can work with `ndarray`. Those familiar with `numpy` should find that the nomenclature and naming conventions of `numpy` are adhered to as closely as possible. I will point out the few differences, where necessary.

Hint:  you can easily port existing `numpy` code, if you `import ulab as np`.

## Initialising an array

A new array can be created by passing either a standard micropython iterable, or another `ndarray` into the constructor.

### Initialising by passing iterables

If the iterable is one-dimensional, i.e., one whose elements are numbers, then a row vector will be created and returned. If the iterable is two-dimensional, i.e., one whose elements are again iterables, a matrix will be created. If the lengths of the iterables is not consistent, a `ValueError` will be raised. Iterables of different types can be mixed in the initialisation function. 

If the `dtype` keyword with the possible `uint8/int8/uint16/int16/float` values is supplied, the new `ndarray` will have that type, otherwise, it assumes `float` as default. 

In [174]:
%%micropython -unix 1

import ulab as np

a = [1, 2, 3, 4, 5, 6, 7, 8]
b = np.array(a)

print("a:\t", a)
print("b:\t", b)

# a two-dimensional array with mixed-type initialisers
c = np.array([range(5), range(20, 25, 1), [44, 55, 66, 77, 88]], dtype=np.uint8)
print("\nc:\t", c)

# and now we throw an exception
d = np.array([range(5), range(10), [44, 55, 66, 77, 88]], dtype=np.uint8)
print("\nd:\t", d)

a:	 [1, 2, 3, 4, 5, 6, 7, 8]
b:	 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)

c:	 array([[0, 1, 2, 3, 4],
	 [20, 21, 22, 23, 24],
	 [44, 55, 66, 77, 88]], dtype=uint8)

Traceback (most recent call last):
  File "/dev/shm/micropython.py", line 15, in <module>
ValueError: iterables are not of the same length



### Initialising by passing arrays

An `ndarray` can be initialised by supplying another array. This statement is almost trivial, since `ndarray`s are iterables themselves.

In [175]:
%%micropython -unix 1

import ulab as np

a = [1, 2, 3, 4, 5, 6, 7, 8]
b = np.array(a)
c = np.array(b)

print("a:\t", a)
print("b:\t", b)
print("\nc:\t", c)

a:	 [1, 2, 3, 4, 5, 6, 7, 8]
b:	 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)

c:	 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)




## Methods of ndarrays

### .asbytearray()

The contents of an `ndarray` can be accessed directly by calling the `.asbytearray` method. This will return a pointer to the underlying `array` object, which can then be manipulated directly.

In [237]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4], dtype=np.int8)
buffer = a.asbytearray()
print("array content:", buffer)
buffer[1] = 123
print("array content:", buffer)

array content: array('b', [1, 2, 3, 4])
array content: array('b', [1, 123, 3, 4])




This in itself wouldn't be very interesting, but since `buffer` is a proper micropython `array`, we can pass it to functions that can employ the buffer protocol. E.g., all the `ndarray` facilities can be applied to the results of timed ADC conversions.

In [ ]:
%%micropython -unix 1

import pyb
import ulab as np

n = 100

adc = pyb.ADC(pyb.Pin.board.X19)
tim = pyb.Timer(6, freq=10)

a = np.array([0]*n, dtype=np.uint8)
buffer = a.asbytearray()
adc.read_timed(buf, tim)

print("ADC results:\t", a)
print("mean of results:\t", np.mean(a))
print("std of results:\t", np.std(a))

Likewise, data can be read directly into `ndarray`s from other interfaces, e.g., SPI, I2C etc. 

### .flatten()

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.htm

## Unary operators

With the exception of `len`, which returns a single number, all unary operators manipulate the underlying data element-wise. 

### len

This operator takes a single argument, and returns either the length (for row vectors), or the number of rows (for matrices) of its argument.

In [105]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4, 5], dtype=np.uint8)
b = np.array([range(5), range(5), range(5), range(5)], dtype=np.uint8)

print("a:\t", a)
print("length of a: ", len(a))
print("shape of a: ", a.shape())
print("\nb:\t", b)
print("length of b: ", len(b))
print("shape of b: ", b.shape())

a:	 array([1, 2, 3, 4, 5], dtype=uint8)
length of a:  5
shape of a:  (1, 5)

b:	 array([[0, 1, 2, 3, 4],
	 [0, 1, 2, 3, 4],
	 [0, 1, 2, 3, 4],
	 [0, 1, 2, 3, 4]], dtype=uint8)
length of b:  4
shape of b:  (4, 5)




 The number returned by `len` is also the length of the iterations, when the array supplies the elements for an iteration (see later).

### invert

The function function is defined for integer data types (`uint8`, `int8`, `uint16`, and `int16`) only, takes a single argument, and returns the element-by-element, bit-wise inverse of the array. If a `float` is supplied, the function raises a `ValueError` exception.

With signed integers (`int8`, and `int16`), the results might be unexpected, as in the example below:

In [98]:
%%micropython -unix 1

import ulab as np

a = np.array([0, -1, -100], dtype=np.int8)
print("a:\t\t", a)
print("inverse of a:\t", ~a)

a = np.array([0, 1, 254, 255], dtype=np.uint8)
print("\na:\t\t", a)
print("inverse of a:\t", ~a)

a:		 array([0, -1, -100], dtype=int8)
inverse of a:	 array([-1, 0, 99], dtype=int8)

a:		 array([0, 1, 254, 255], dtype=uint8)
inverse of a:	 array([255, 254, 1, 0], dtype=uint8)




### abs

This function takes a single argument, and returns the element-by-element absolute value of the array. When the data type is unsigned (`uint8`, or `uint16`), a copy of the array will be returned immediately, and no calculation takes place.

In [73]:
%%micropython -unix 1

import ulab as np

a = np.array([0, -1, -100], dtype=np.int8)
print("a:\t\t\t ", a)
print("absolute value of a:\t ", abs(a))

a:			  array([0, -1, -100], dtype=int8)
absolute value of a:	  array([0, 1, 100], dtype=int8)




### neg

This operator takes a single argument, and changes the sign of each element in the array. Unsigned values are wrapped. 

In [99]:
%%micropython -unix 1

import ulab as np

a = np.array([10, -1, 1], dtype=np.int8)
print("a:\t\t", a)
print("negative of a:\t", -a)

b = np.array([0, 100, 200], dtype=np.uint8)
print("\nb:\t\t", b)
print("negative of b:\t", -b)

a:		 array([10, -1, 1], dtype=int8)
negative of a:	 array([-10, 1, -1], dtype=int8)

b:		 array([0, 100, 200], dtype=uint8)
negative of b:	 array([0, 156, 56], dtype=uint8)




### pos

This function takes a single argument, and simply returns a copy of the array.

In [85]:
%%micropython -unix 1

import ulab as np

a = np.array([10, -1, 1], dtype=np.int8)
print("a:\t\t", a)
print("positive of a:\t", +a)

a:		 array([10, -1, 1], dtype=int8)
positive of a:	 array([10, -1, 1], dtype=int8)




## Binary operations

All binary operators work element-wise. This also means that the operands either must have the same shape, or one of them must be a scalar.

**WARNING:** `numpy` also allows operations between a matrix, and a row vector, if the row vector has exactly as many elements, as many columns the matrix has. This feature will be added in future versions of `ulab`.

In [188]:
a = array([[1, 2, 3], [4, 5, 6], [7, 8, 6]])
b = array([10, 20, 30])
a+b

array([[11, 22, 33],
       [14, 25, 36],
       [17, 28, 36]])

### Upcasting

Binary operations require special attention, because two arrays with different typecodes can be the operands of an operation, in which case it is not trivial, what the typecode of the result is. This decision on the result's typecode is called upcasting. Since the number of typecodes in `ulab` is significantly smaller than that of `numpy`, we have to define new upcasting rules. Where possible, I followed `numpy`'s conventions. 

`ulab` observes the following upcasting rules:

1. Operations with two `ndarray`s of the same `dtype` preserve their `dtype`, even when the results overflow.

2. if either of the operands is a float, the result is automatically a float

3. 
    
| left hand side | right hand side | result |
|----------------|-----------------|--------|
|`uint8`         |`int8`           |`int16` |
|`uint8`         |`int16`          |`int16` |
|`uint8`         |`uint16`         |`uint16`|
|`int8`          |`int16`          |`int16` |
|`int8`          |`uint16`         |`uint16`|
|`uint16`        |`int16`          |`float` |
    
Note that the last two operations are promoted to `int32` in `numpy`.
    
4. When the right hand side of a binary operator is a micropython variable, `mp_obj_int`, or `mp_obj_float`, then the result will be promoted to `dtype` `float`. This is necessary, because a micropython integer can be 31 bites wide. Other micropython types (e.g., lists, tuples, etc.) raise a `TypeError` exception. 

**WARNING:** Due to the lower number of available data types, the upcasting rules of `ulab` are slightly different to those of `numpy`. Keep this in mind, when porting code!

Upcasting can be seen in action in the following snippet:

In [212]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4], dtype=np.uint8)
b = np.array([1, 2, 3, 4], dtype=np.int8)
print("a:\t", a)
print("b:\t", b)
print("a+b:\t", a+b)

c = np.array([1, 2, 3, 4], dtype=np.float)
print("\na:\t", a)
print("c:\t", c)
print("a*c:\t", a*c)

a:	 array([1, 2, 3, 4], dtype=uint8)
b:	 array([1, 2, 3, 4], dtype=int8)
a+b:	 array([2, 4, 6, 8], dtype=int16)

a:	 array([1, 2, 3, 4], dtype=uint8)
c:	 array([1.0, 2.0, 3.0, 4.0], dtype=float)
a*c:	 array([1.0, 4.0, 9.0, 16.0], dtype=float)




**WARNING:** If a binary operation involves an `ndarray` and a micropython type (integer, or float), then the array must be on the left hand side. 

In [181]:
%%micropython -unix 1

import ulab as np

# this is going to work
a = np.array([1, 2, 3, 4], dtype=np.uint8)
b = 12
print("a:\t", a)
print("b:\t", b)
print("a+b:\t", a+b)

# but this will spectacularly fail
print("b+a:\t", b+a)

a:	 array([1, 2, 3, 4], dtype=uint8)
b:	 12
a+b:	 array([13, 14, 15, 16], dtype=uint8)

Traceback (most recent call last):
  File "/dev/shm/micropython.py", line 12, in <module>
TypeError: unsupported types for __add__: 'int', 'ndarray'



The reason for this lies in how micropython resolves binary operators, and this means that a fix can only be implemented, if micropython itself changes the corresponding function(s). Till then, keep `ndarray`s on the left hand side. 

### Benchmarks

The following snippet compares the performance of binary operations to a possible implementation in python.

In [ ]:
%%micropython -unix 1

import ulab as np

@timeit
def py_add(a, b):
    return [a[i]+b[i] for i in range(len(a))]

@timeit
def py_multiply(a, b):
    return [a[i]*b[i] for i in range(len(a))]

@timeit
def ulab_add(a, b):
    return a + b

@timeit
def ulab_multiply(a, b):
    return a * b

a = [0.0]*1000
b = range(1000)

py_add(a, b)

py_multiply(a, b)

a = np.linspace(0, 10, 1000)
b = np.ones(1000)

ulab_add(a, b)

ulab_multiply(a, b)

## Comparison operators

The smaller than, greater than, smaller or equal, and greater or equal operators return a vector of Booleans indicating the positions (`True`), where the condition is satisfied. 

In [118]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.uint8)
print(a < 5)

[True, True, True, True, False, False, False, False]




**WARNING:** Note that `numpy` returns an array of Booleans. For most use cases this fact should not make a difference. 

In [119]:
a = array([1, 2, 3, 4, 5, 6, 7, 8])
a < 5

array([ True,  True,  True,  True, False, False, False, False])

These operators work with matrices, too, in which case a list of lists of Booleans will be returned:

In [122]:
%%micropython -unix 1

import ulab as np

a = np.array([range(0, 5, 1), range(1, 6, 1), range(2, 7, 1)], dtype=np.uint8)
print(a)
print(a < 5)

array([[0, 1, 2, 3, 4],
	 [1, 2, 3, 4, 5],
	 [2, 3, 4, 5, 6]], dtype=uint8)
[[True, True, True, True, True], [True, True, True, True, False], [True, True, True, False, False]]




## Iterating over arrays

`ndarray`s are iterable, which means that their elements can also be accessed as can the elements of a list, tuple, etc. If the array is one-dimensional, the iterator returns scalars, otherwise a new one-dimensional `ndarray`, which is simply a copy of the corresponding row of the matrix, i.e, its data type will be inherited.

In [104]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4, 5], dtype=np.uint8)
b = np.array([range(5), range(10, 15, 1), range(20, 25, 1), range(30, 35, 1)], dtype=np.uint8)

print("a:\t", a)

for i, _a in enumerate(a):
    print("element %d in a:"%i, _a)
    
print("\nb:\t", b)

for i, _b in enumerate(b):
    print("element %d in b:"%i, _b)

a:	 array([1, 2, 3, 4, 5], dtype=uint8)
element 0 in a: 1
element 1 in a: 2
element 2 in a: 3
element 3 in a: 4
element 4 in a: 5

b:	 array([[0, 1, 2, 3, 4],
	 [10, 11, 12, 13, 14],
	 [20, 21, 22, 23, 24],
	 [30, 31, 32, 33, 34]], dtype=uint8)
element 0 in b: array([0, 1, 2, 3, 4], dtype=uint8)
element 1 in b: array([10, 11, 12, 13, 14], dtype=uint8)
element 2 in b: array([20, 21, 22, 23, 24], dtype=uint8)
element 3 in b: array([30, 31, 32, 33, 34], dtype=uint8)




## Slicing and indexing

Copies of sub-arrays can be created by indexing, and slicing.

### Indexing

The simplest form of indexing is specifying a single integer between the square brackets as in 

In [116]:
%%micropython -unix 1

import ulab as np

a = np.array(range(10), dtype=np.uint8)
print("a:\t\t\t\t\t\t", a)
print("the first, and first from right element of a:\t", a[0], a[-1])
print("the second, and second from right element of a:\t", a[1], a[-2])

a:						 array([0, 1, 2, ..., 7, 8, 9], dtype=uint8)
the first, and first from right element of a:	 0 9
the second, and second from right element of a:	 1 8




Indices are (not necessarily non-negative) integers, or a list of Booleans. By using Boolean list, we can select those elements of an array that satisfy a specific condition. At the moment, such indexing is defined for row vectors only, for matrices the function raises a `ValueError` exception, though this will be rectified in a future version of `ulab`.

In [183]:
%%micropython -unix 1

import ulab as np

a = np.array(range(9), dtype=np.float)
print("a:\t", a)
print("a < 5:\t", a[a < 5])

a:	 array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)
a < 5:	 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)




Indexing with Boolean arrays can take more complicated expressions. This is a very concise way of comparing two vectors, e.g.:

In [184]:
%%micropython -unix 1

import ulab as np

a = np.array(range(9), dtype=np.uint8)
b = np.array([4, 4, 4, 3, 3, 3, 13, 13, 13], dtype=np.uint8)
print("a:\t", a)
print("a*2:\t", a*a)
print("b:\t", b)
print("100*sin(b):\t", np.sin(b)*100.0)
print("a < b:\t", a[a*a > np.sin(b)*100.0])

a:	 array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=uint8)
a*2:	 array([0, 1, 4, 9, 16, 25, 36, 49, 64], dtype=uint8)
b:	 array([4, 4, 4, 3, 3, 3, 13, 13, 13], dtype=uint8)
100*sin(b):	 array([-75.68025207519532, -75.68025207519532, -75.68025207519532, 14.11200046539307, 14.11200046539307, 14.11200046539307, 42.01670455932617, 42.01670455932617, 42.01670455932617], dtype=float)
a < b:	 array([0, 1, 2, 4, 5, 7, 8], dtype=uint8)




### Slicing

You can also generate sub-arrays by specifying slices as the index of an array. Slices are special python objects of the form 

```python
slice = start:end:stop
```
where `start`, `end`, and `stop` are (not necessarily non-negative) integers. Not all of these three numbers must be specified in an index, in fact, all three of them can be missing. The interpreter takes care of filling in the missing values. (Note that slices cannot be defined in this way, only there, where an index is expected.) For a good explanation on how slices work in python, you can read the stackoverflow question https://stackoverflow.com/questions/509211/understanding-slice-notation.

# Universal functions

Standard mathematical functions can be calculated on any iterable, and on `ndarray`s without having to change the call signature. In all cases the functions return a new `ndarray` of typecode `float` (since these functions usually generate float values, anyway). The functions execute faster with `ndarray` arguments than with iterables, because the values of the input vector can be extracted faster. 

In [167]:
%%micropython -unix 1

import ulab as np

a = range(9)
b = np.array(a)

# works with ranges, lists, tuples etc.
print('a:\t', a)
print('exp(a):\t', np.exp(a))

# with 1D arrays
print('\nb:\t', b)
print('exp(b):\t', np.exp(b))

# as well as with matrices
c = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print('\nc:\t', c)
print('exp(c):\t', np.exp(c))

a:	 range(0, 9)
exp(a):	 array([1.0, 2.718281745910645, 7.389056205749512, 20.08553695678711, 54.59814834594727, 148.4131622314453, 403.4288024902343, 1096.633178710938, 2980.9580078125], dtype=float)

b:	 array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)
exp(b):	 array([1.0, 2.718281745910645, 7.389056205749512, 20.08553695678711, 54.59814834594727, 148.4131622314453, 403.4288024902343, 1096.633178710938, 2980.9580078125], dtype=float)

c:	 array([[1.0, 2.0, 3.0],
	 [4.0, 5.0, 6.0],
	 [7.0, 8.0, 9.0]], dtype=float)
exp(c):	 array([[2.718281745910645, 7.389056205749512, 20.08553695678711],
	 [54.59814834594727, 148.4131622314453, 403.4288024902343],
	 [1096.633178710938, 2980.9580078125, 8103.083984375001]], dtype=float)




The overhead for iterables is

In [ ]:
%%micropython -unix 1

import ulab as np

a = [0]*1000
b = np.array(a)

def measure_run_time(x):
    return np.exp(x)

@timeit(n=1000)
measure_run_time(a)

@timeit(n=1000)
measure_run_time(b)

Of course, such a time saving is reasonable only, if the data are already available as an `ndarray`. If one has to initialise the `ndarray` from the list, then there is no gain, because the iterator was simple pushed into the initialisation function.

# Numerical

## linspace

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html

## argmin, argmax

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmin.html

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

Difference to `numpy`: the `out` keyword argument is not implemented.

## roll

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.roll.html

The roll function shifts the content of a vector by the positions given as the second argument. If the `axis` keyword is supplied, the shift is applied to the given axis.

In [229]:
%%micropython -unix 1

import ulab as np

a = np.array([1, 2, 3, 4, 5, 6, 7, 8])
print("a:\t\t\t", a)

np.roll(a, 2)
print("a rolled to the left:\t", a)

# this should be the original vector
np.roll(a, -2)
print("a rolled to the right:\t", a)

a:			 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)
a rolled to the left:	 array([3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 1.0, 2.0], dtype=float)
a rolled to the right:	 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=float)




Rolling works with matrices, too:

In [245]:
%%micropython -unix 1

import ulab as np

a = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print("a:\n", a)

np.roll(a, 2)
print("\na rolled to the left:\n", a)

np.roll(a, -1, axis=1)
print("a rolled to the up:\t", a)

a:
 array([[1.0, 2.0, 3.0, 4.0],
	 [5.0, 6.0, 7.0, 8.0]], dtype=float)

a rolled to the left:
 array([[3.0, 4.0, 1.0, 2.0],
	 [7.0, 8.0, 5.0, 6.0]], dtype=float)
a rolled to the up:	 array([[7.0, 8.0, 5.0, 6.0],
	 [3.0, 4.0, 1.0, 2.0]], dtype=float)




### Simple running weighted average

As a demonstration of the conciseness of `ulab/numpy` operations, we will calculate an exponentially weighted running average of a measurement vector in just a couple of lines. I chose this particular example, because I think that this can indeed be used in real-life applications.

In [230]:
%%micropython -unix 1

import ulab as np

def dummy_adc():
    # dummy adc function, so that the results are reproducible
    return 2
    
n = 10
# These are the normalised weights; the last entry is the most dominant
weight = np.exp([1, 2, 3, 4, 5])
weight = weight/np.sum(weight)

print(weight)
# initial array of samples
samples = np.array([0]*n)

for i in range(n):
    # a new datum is inserted on the right hand side. This simply overwrites whatever was in the last slot
    samples[-1] = dummy_adc()
    print(np.mean(samples[-5:]*weight))
    print(samples[-5:])
    # the data are shifted by one position to the left
    np.roll(samples, 1)

array([0.01165623031556606, 0.03168492019176483, 0.08612854033708572, 0.234121635556221, 0.6364086270332336], dtype=float)
0.2545634508132935
array([0.0, 0.0, 0.0, 0.0, 2.0], dtype=float)
0.3482121050357819
array([0.0, 0.0, 0.0, 2.0, 2.0], dtype=float)
0.3826635211706161
array([0.0, 0.0, 2.0, 2.0, 2.0], dtype=float)
0.3953374892473221
array([0.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)
0.3999999813735485
array([2.0, 2.0, 2.0, 2.0, 2.0], dtype=float)




# Linalg

## transpose

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.transpose.html

## reshape

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html

## ones, zeros

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ones.html

A couple of special arrays and matrices can easily be initialised by calling one of the `ones`, or `zeros` functions. `ones` and `zeros` follow the same pattern, and have the call signature

```python
ones(shape, dtype=float)
zeros(shape, dtype=float)
```
where shape is either an integer, or a 2-tuple.

In [14]:
%%micropython -unix 1

import ulab as np

print(np.ones(6, dtype=np.uint8))
print(np.zeros((6, 4)))

array([1, 1, 1, 1, 1, 1], dtype=uint8)
array([[0.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0]], dtype=float)




## eye

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.eye.html

Another special array method is the `eye` function, whose call signature is 

```python
eye(N, M, k=0, dtype=float)
```
where `N` (`M`) specify the dimensions of the matrix (if only `N` is supplied, then we get a square matrix, otherwise one with `N` rows, and `M` columns), whether the ones are in the main diagonal (`k=0`), or in another diagonal that is shifted by `k` positions. Here are a couple of examples.

### With a single argument

In [23]:
%%micropython -unix 1

import ulab as np

print(np.eye(5))

array([[1.0, 0.0, 0.0, 0.0, 0.0],
	 [0.0, 1.0, 0.0, 0.0, 0.0],
	 [0.0, 0.0, 1.0, 0.0, 0.0],
	 [0.0, 0.0, 0.0, 1.0, 0.0],
	 [0.0, 0.0, 0.0, 0.0, 1.0]], dtype=float)




### Specifying the dimensions of the matrix

In [21]:
%%micropython -unix 1

import ulab as np

print(np.eye(4, M=6, dtype=np.int8))

array([[1, 0, 0, 0],
	 [0, 1, 0, 0],
	 [0, 0, 1, 0],
	 [0, 0, 0, 1],
	 [0, 0, 0, 0],
	 [0, 0, 0, 0]], dtype=int8)




### Shifting the diagonal

In [22]:
%%micropython -unix 1

import ulab as np

print(np.eye(4, M=6, k=-1, dtype=np.int16))

array([[0, 0, 0, 0],
	 [1, 0, 0, 0],
	 [0, 1, 0, 0],
	 [0, 0, 1, 0],
	 [0, 0, 0, 1],
	 [0, 0, 0, 0]], dtype=int16)




## inv

A square matrix, provided that it is not singular, can be inverted by calling the `inv` function that takes a single argument. The inversion is based on successive elimination, and raises a `ValueError` exception, if the matrix turns out to be singular.

In [31]:
%%micropython -unix 1

import ulab as np

m = np.array([[1, 2, 3], [4, 5, 6], [7, 8.5, 9]])

print(np.inv(m))

array([[-2.000000238418579, 2.5, -1.0],
	 [2.000000238418579, -4.0, 2.0],
	 [-0.3333334922790527, 1.833333373069763, -1.0]], dtype=float)




Note that the cost of inverting a matrix is approximately is twice as many floats (RAM), as the number of entries in the original matrix, and approximately as many operations, as the number of entries.

## dot

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html

Once you can invert a matrix, you might want to know, whether the inversion is correct. You can simply take the original matrix and its inverse, and multiply them by calling the `dot` function, which takes the two matrices as its arguments. If the matrix dimensions do not match, the function raises a `ValueError`. The result of the multiplication is expected to be the unit matrix, which is demonstrated below.

In [249]:
%%micropython -unix 1

import ulab as np

m = np.array([[1, 2, 3], [4, 5, 6], [7, 10, 9]], dtype=np.uint8)
n = np.inv(m)
print("m:\t", m)
print("m^-1:\t", n)
# this should be the unit matrix
print("m*m^-1:\t", np.dot(m, n))

m:	 array([[1, 2, 3],
	 [4, 5, 6],
	 [7, 10, 9]], dtype=uint8)
m^-1:	 array([[-1.25000011920929, 1.0, -0.25],
	 [0.5000000596046448, -1.0, 0.5],
	 [0.4166666269302368, 0.3333333432674408, -0.25]], dtype=float)
m*m^-1:	 array([[0.9999998807907104, 0.0, 0.0],
	 [-4.76837158203125e-07, 1.0, 0.0],
	 [-9.5367431640625e-07, 0.0, 1.0]], dtype=float)




Note that for matrix multiplication you don't necessarily need square matrices, it is enough, if their dimensions are compatible (i.e., the the left-hand-side matrix has as many columns, as does the right-hand-side matrix rows):

In [37]:
%%micropython -unix 1

import ulab as np

m = np.array([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=np.uint8)
n = np.array([[1, 2], [3, 4], [5, 6], [7, 8]], dtype=np.uint8)
print(m)
print(n)
print(np.dot(m, n))

array([[1, 2, 3, 4],
	 [5, 6, 7, 8]], dtype=uint8)
array([[1, 2],
	 [3, 4],
	 [5, 6],
	 [7, 8]], dtype=uint8)
array([[7.0, 10.0],
	 [23.0, 34.0]], dtype=float)




## det

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html

The `det` function takes a single argument, and calculates the determinant of a square matrix that is not singular. The calculation is based on successive elimination of the matrix elements, and the function raises a `ValueError` exception, if the matrix is singular.

**WARNING:** this function is to be implemented.

## eig

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html

The `eig` function calculates the eigenvalues and the eigenvectors of a real, symmetric square matrix. It takes a single argument, and returns a tuple with the eigenvalues, and eigenvectors. With the help of the eigenvectors, amongst other things, you can implement sophisticated stabilisation routines for robots.

**WARNING:** this function is to be implemented.

# Polynomials

## polyval

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyval.html

polyval takes two arguments, both arrays or other iterables, and returns the 

In [253]:
%%micropython -unix 1

import ulab as np

p = [1, 1, 1, 0]
x = [0, 1, 2, 3, 4]
print('coefficients: ', p)
print('independent values: ', x)
print('\nvalues of p(x): ', np.polyval(p, x))

# the same works with one-dimensional ndarrays
a = np.array(x)
print('\nndarray (a): ', a)
print('value of p(a): ', np.polyval(p, a))

coefficients:  [1, 1, 1, 0]
independent values:  [0, 1, 2, 3, 4]

values of p(x):  array([0.0, 3.0, 14.0, 39.0, 84.0], dtype=float)

ndarray (a):  array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
value of p(a):  array([0.0, 3.0, 14.0, 39.0, 84.0], dtype=float)




## polyfit

numpy: https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html

polyfit takes two, or three arguments. The last one is the degree of the polynomial that will be fitted, the last but one is an array or iterable with the `y` values, and the first one, an array or iterable with the `x` values, can be dropped. If that is the case, `x` will be generated in the function, assuming uniform sampling. 

If the length of `x`, and `y` are not the same, the function raises a `ValueError`.

In [259]:
%%micropython -unix 1

import ulab as np

x = np.array([-3, -2, -1, 0, 1, 2, 3])
y = np.array([9, 4, 1, 0, 1, 4, 9])
print('independent values:\t', x)
print('dependent values:\t', y)
print('fitted values:\t\t', np.polyfit(x, y, 2))

# the same with missing x
print('\ndependent values:\t', y)
print('fitted values:\t\t', np.polyfit(y, 2))

independent values:	 array([-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0], dtype=float)
dependent values:	 array([9.0, 4.0, 1.0, 0.0, 1.0, 4.0, 9.0], dtype=float)
fitted values:		 array([1.00000011920929, 0.0, 0.0], dtype=float)

dependent values:	 array([9.0, 4.0, 1.0, 0.0, 1.0, 4.0, 9.0], dtype=float)
fitted values:		 array([0.9999995231628418, -6.0, 8.999998092651367], dtype=float)




# Fourier transforms

In [ ]:
%%micropython -unix 1

import ulab as np

x = np.linspace(0, 10, num=1024)
y = np.sin(x)

@timeit
def np_fft(y)
    return np.fft(y)

a, b = np_fft(y)

A comment on the speed: a 1024-point transform implemented in python would cost around 90 ms, and 13 ms in assembly, if the code runs on the pyboard, v.1.1. You can gain a factor of four by moving to the D series 
https://github.com/peterhinch/micropython-fourier/blob/master/README.md#8-performance. 

The C implementation runs in less than 2 ms on the pyboard (we have just measured that), and has been reported to run in under 0.8 ms on the D series board. That is a factor of more than four improvement. 